In [27]:
def cal_area_theta(center_a,center_b,d,theta_l,theta_r,R):

    a = center_a
    b = center_b
    hl = R*tf.sin(theta_l)+center_b
    hr = R*tf.sin(theta_r)+center_b
    r = R

    theta = theta_l - theta_r

    point_1 = (hl * a) / (hl - b)

    point_2 = d + (a - d) * hr / (hr - b)

    fan_shape = 0.5 * theta * (pow(r,2))

    triangle_1 = 0.5 * point_1 * hl

    triangle_2 = 0.5 * (d - point_2) * hr

    triangle_3 = 0.5 * (point_2 - point_1) * b

    area = fan_shape + triangle_1 + triangle_2 + triangle_3

    # print(f'theta_1: {math.degrees(theta_1)} theta_2: {math.degrees(theta_2)} theta: {math.degrees(theta)}')

    return area

In [28]:
# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np

# Set data type
DTYPE='float64'
tf.keras.backend.set_floatx(DTYPE)

# Define contact line velocity
# theta_0 : equilibrium contact angle
# theta_d : dynamic contact angle
def fun_contact_line_velocity( f_c,i_t,theta_0, theta_d ):
    one = tf.constant(1, dtype=tf.float64)
    return -f_c*i_t*(-tf.cos(theta_0)+tf.cos(theta_d))

# Set constants
pi = tf.constant(np.pi, dtype=tf.float64)
p_f_c = tf.constant(-2.02, dtype=tf.float64) # positive_friction_coefficient
n_f_c = tf.constant(-5.64, dtype=tf.float64) # nagative_friction_coefficient
i_t = tf.constant(7.6408/1000, dtype=tf.float64) #interfacial tension between the two liquids
d = tf.constant(0.321/1000, dtype=tf.float64) #gab

thetal_0 = tf.constant(np.deg2rad(94), dtype=tf.float64)
thetal_d = tf.constant(np.deg2rad(55), dtype=tf.float64)
thetar_0 = tf.constant(np.deg2rad(59), dtype=tf.float64)
thetar_d = tf.constant(np.deg2rad(109), dtype=tf.float64)

vl = tf.constant(fun_contact_line_velocity(n_f_c ,i_t,thetal_0, thetal_d ), dtype=tf.float64)
vr = tf.constant(fun_contact_line_velocity(p_f_c ,i_t,thetar_0 , thetar_d ), dtype=tf.float64)

# initial_data = [a0 ,b0,r0 ,theta_l0,theta_r0 ]
# cal_area_theta(center_a,center_b,d,theta_l,theta_r,R):
initial_data = [-0.0000566758,-0.0006874250504951296,.0009033011295732994,1.500983153589793,1.0874056000]
initial_area = cal_area_theta(-0.0000566758,-0.0006874250504951296,0.000321,1.500983153589793,1.0874056000,0.0009033011295732994)

# Define residual of the condition
def fun_r(yl_t , yr_t ,vl,vr,area_t):
    return (yl_t-vl) + (yr_t-vr) + area_t

In [14]:
print(vl)
print(vr)


tf.Tensor(0.027723860480927242, shape=(), dtype=float64)
tf.Tensor(-0.012974266236244947, shape=(), dtype=float64)


In [17]:
print(vr)
print(vl)

tf.Tensor(-0.012974266236244947, shape=(), dtype=float64)
tf.Tensor(0.027723860480927242, shape=(), dtype=float64)


1.500983153589793


In [29]:
import tensorflow as tf
import numpy as np

In [30]:
# time interval
time = [0.0,0.0280]
# start_time
start_time= tf.constant(time[0], shape=(1,1),dtype=DTYPE)
# last_time
end_time = tf.constant(time[1], shape=(1,1),dtype=DTYPE)

# Set random seed for reproducible results
tf.random.set_seed(0)

t_data = tf.random.uniform(shape=[1,10000],minval=0.0,maxval=0.0280, dtype=DTYPE)
t_data = tf.reshape(t_data,shape=(-1,1))

#t_data = tf.linspace(0.0,0.0280,100)
#t_data = tf.reshape(t_data,shape=(-1,1))
#
# t_data =tf.concat([start_time,t_data],axis=0)
# t_data = tf.concat([t_data,end_time],axis=0)
# t_data = tf.constant(t_data,dtype=DTYPE)
print(t_data)

tf.Tensor(
[[0.00940245]
 [0.00792753]
 [0.00933509]
 ...
 [0.02178978]
 [0.00309513]
 [0.02021461]], shape=(10000, 1), dtype=float64)


In [31]:
def init_model(num_hidden_layers=8, num_neurons_per_layer=20):
    # Initialize a feedforward neural network
    model = tf.keras.Sequential()

    # Input is one-dimensional (time)
    model.add(tf.keras.Input(1))

    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons_per_layer,
            activation=tf.keras.activations.get('tanh'),
            kernel_initializer='glorot_normal'))

    # Output is one-dimensional (center_a , center_b , R , theat_l , thetah_r)
    model.add(tf.keras.layers.Dense(5))

    return model

In [47]:
test_data=tf.linspace(0.0, 0.0280, 281)
print(test_data)
test_data = tf.reshape(test_data,shape=(-1,1))

tf.Tensor(
[0.     0.0001 0.0002 0.0003 0.0004 0.0005 0.0006 0.0007 0.0008 0.0009
 0.001  0.0011 0.0012 0.0013 0.0014 0.0015 0.0016 0.0017 0.0018 0.0019
 0.002  0.0021 0.0022 0.0023 0.0024 0.0025 0.0026 0.0027 0.0028 0.0029
 0.003  0.0031 0.0032 0.0033 0.0034 0.0035 0.0036 0.0037 0.0038 0.0039
 0.004  0.0041 0.0042 0.0043 0.0044 0.0045 0.0046 0.0047 0.0048 0.0049
 0.005  0.0051 0.0052 0.0053 0.0054 0.0055 0.0056 0.0057 0.0058 0.0059
 0.006  0.0061 0.0062 0.0063 0.0064 0.0065 0.0066 0.0067 0.0068 0.0069
 0.007  0.0071 0.0072 0.0073 0.0074 0.0075 0.0076 0.0077 0.0078 0.0079
 0.008  0.0081 0.0082 0.0083 0.0084 0.0085 0.0086 0.0087 0.0088 0.0089
 0.009  0.0091 0.0092 0.0093 0.0094 0.0095 0.0096 0.0097 0.0098 0.0099
 0.01   0.0101 0.0102 0.0103 0.0104 0.0105 0.0106 0.0107 0.0108 0.0109
 0.011  0.0111 0.0112 0.0113 0.0114 0.0115 0.0116 0.0117 0.0118 0.0119
 0.012  0.0121 0.0122 0.0123 0.0124 0.0125 0.0126 0.0127 0.0128 0.0129
 0.013  0.0131 0.0132 0.0133 0.0134 0.0135 0.0136 0.0137 0.0138 0.

In [50]:
print(np.rad2deg(1.09016884e+00))

62.46207348867272


In [48]:
print(test_data)

tf.Tensor(
[[0.    ]
 [0.0001]
 [0.0002]
 [0.0003]
 [0.0004]
 [0.0005]
 [0.0006]
 [0.0007]
 [0.0008]
 [0.0009]
 [0.001 ]
 [0.0011]
 [0.0012]
 [0.0013]
 [0.0014]
 [0.0015]
 [0.0016]
 [0.0017]
 [0.0018]
 [0.0019]
 [0.002 ]
 [0.0021]
 [0.0022]
 [0.0023]
 [0.0024]
 [0.0025]
 [0.0026]
 [0.0027]
 [0.0028]
 [0.0029]
 [0.003 ]
 [0.0031]
 [0.0032]
 [0.0033]
 [0.0034]
 [0.0035]
 [0.0036]
 [0.0037]
 [0.0038]
 [0.0039]
 [0.004 ]
 [0.0041]
 [0.0042]
 [0.0043]
 [0.0044]
 [0.0045]
 [0.0046]
 [0.0047]
 [0.0048]
 [0.0049]
 [0.005 ]
 [0.0051]
 [0.0052]
 [0.0053]
 [0.0054]
 [0.0055]
 [0.0056]
 [0.0057]
 [0.0058]
 [0.0059]
 [0.006 ]
 [0.0061]
 [0.0062]
 [0.0063]
 [0.0064]
 [0.0065]
 [0.0066]
 [0.0067]
 [0.0068]
 [0.0069]
 [0.007 ]
 [0.0071]
 [0.0072]
 [0.0073]
 [0.0074]
 [0.0075]
 [0.0076]
 [0.0077]
 [0.0078]
 [0.0079]
 [0.008 ]
 [0.0081]
 [0.0082]
 [0.0083]
 [0.0084]
 [0.0085]
 [0.0086]
 [0.0087]
 [0.0088]
 [0.0089]
 [0.009 ]
 [0.0091]
 [0.0092]
 [0.0093]
 [0.0094]
 [0.0095]
 [0.0096]
 [0.0097]
 [0.0098]

tf.Tensor([[0.]], shape=(1, 1), dtype=float32)
tf.Tensor([[0. 0. 0. 0. 0.]], shape=(1, 5), dtype=float32)


In [32]:
# Output is one-dimensional (center_a , center_b , R , theta_l , theta_r)
def get_r(model, t_data):
    #t_data = tf.reshape(t_data ,shape=(-1,1))
    #t_data= tf.constant(t_data)
    #t_data = tf.constant(t_data,dtype=DTYPE)
    #n_f_c = tf.constant(-5.64, dtype=DTYPE) # nagative_friction_coefficient
    #i_t = tf.constant(7.64, dtype=DTYPE) #interfacial tension between the two liquids
    #vl = tf.constant(fun_contact_line_velocity(n_f_c ,i_t,1.6406095000, 0.9745209500), dtype=DTYPE)
    #vr = tf.constant(fun_contact_line_velocity(n_f_c ,i_t,1.0874056000, 1.8590153000), dtype=DTYPE)

    # A tf.GradientTape is used to compute derivatives in TensorFlow
    with tf.GradientTape(persistent=True) as tape:
        # Split t and x to compute partial derivatives
        t = t_data

        # Variables t and x are watched during tape
        tape.watch(t)

        # Determine residual
        u = model(t_data)
        a = tf.reshape(u[:,0],shape=(-1,1))
        b = tf.reshape(u[:,1],shape=(-1,1))
        R = tf.reshape(u[:,2],shape=(-1,1))
        theta_l =tf.reshape(u[:,3],shape=(-1,1))
        theta_r = tf.reshape(u[:,4],shape=(-1,1))

        yl = R*tf.sin(theta_l)+b
        yr = R*tf.sin(theta_r)+b
        area = cal_area_theta(a,b,d,theta_l,theta_r,R)

    yl_t = tape.gradient(yl, t)
    yr_t = tape.gradient(yr, t)
    area_t = tape.gradient(area, t)

    print(yl_t)
    print(yr_t)
    print(area_t )

    del tape

    return fun_r(yl_t , yr_t ,vl ,vr,area_t)

tf.Tensor(
[[-1.07748329e-03  6.54141791e-03  3.12457047e-03 -3.89681011e-03
  -8.59190302e-04]
 [-7.62303243e-04  4.62802267e-03  2.21061660e-03 -2.75695627e-03
  -6.07853872e-04]
 [-1.97569863e-03  1.19936541e-02  5.72890416e-03 -7.14499829e-03
  -1.57553260e-03]
 [-2.07114080e-03  1.25729330e-02  6.00560289e-03 -7.49012223e-03
  -1.65165926e-03]
 [-1.53764198e-03  9.33475979e-03  4.45884187e-03 -5.56091545e-03
  -1.22615788e-03]
 [-2.98086531e-03  1.80932898e-02  8.64251703e-03 -1.07793380e-02
  -2.37738690e-03]
 [-1.82011351e-03  1.10493265e-02  5.27783111e-03 -6.58238959e-03
  -1.45143922e-03]
 [-3.68286180e-03  2.23515872e-02  1.06766261e-02 -1.33169778e-02
  -2.93759489e-03]
 [-2.57100863e-03  1.56064443e-02  7.45461695e-03 -9.29753203e-03
  -2.05039675e-03]
 [-4.62674681e-04  2.80897412e-03  1.34173152e-03 -1.67332590e-03
  -3.68930923e-04]
 [-2.61927163e-03  1.58993118e-02  7.59451510e-03 -9.47203394e-03
  -2.08889856e-03]
 [-2.44438346e-03  1.48380641e-02  7.08758691e-03 -8.8

In [33]:
def compute_loss(model,t_data,initial_data): #initial_data = [a0 ,b0,r0 ,theta_l0,theta_r0 ]
    t_0 = tf.zeros((1,1))

    # Compute phi^r
    r = get_r(model, t_data)
    phi_r = tf.reduce_mean(tf.square(r))

    # Initialize loss
    loss = phi_r

    # Add phi^0 and phi^b to the loss
    u = model(t_0)
    pred_a = tf.reshape(u[:,0],shape=(-1,1))
    pred_b = tf.reshape(u[:,1],shape=(-1,1))
    pred_R = tf.reshape(u[:,2],shape=(-1,1))
    pred_theta_l =tf.reshape(u[:,3],shape=(-1,1))
    pred_theta_r = tf.reshape(u[:,4],shape=(-1,1))

    loss += tf.reduce_mean(tf.square(initial_data[0] - pred_a))
    loss += tf.reduce_mean(tf.square(initial_data[1] - pred_b))
    loss += tf.reduce_mean(tf.square(initial_data[2] - pred_R))
    loss += tf.reduce_mean(tf.square(initial_data[3] - pred_theta_l))
    loss += tf.reduce_mean(tf.square(initial_data[4] - pred_theta_r))

    return loss


In [34]:
def get_grad(model,t_data,initial_data):

    with tf.GradientTape(persistent=True) as tape:
        # This tape is for derivatives with
        # respect to trainable variables
        tape.watch(model.trainable_variables)
        loss = compute_loss(model,t_data,initial_data)

    g = tape.gradient(loss, model.trainable_variables)
    del tape

    return loss, g

In [35]:
# Initialize model aka u_\theta
model = init_model()

# We choose a piecewise decay of the learning rate, i.e., the
# step size in the gradient descent type algorithm
# the first 1000 steps use a learning rate of 0.01
# from 1000 - 3000: learning rate = 0.001
# from 3000 onwards: learning rate = 0.0005

lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay([1000,3000],[1e-2,1e-3,5e-4])

# Choose the optimizer
optim = tf.keras.optimizers.Adam(learning_rate=lr)

In [36]:
from time import time

# Define one training step as a TensorFlow function to increase speed of training
@tf.function
def train_step(initial_data):
    # Compute current loss and gradient w.r.t. parameters
    loss, grad_theta = get_grad(model,t_data,initial_data)

    # Perform gradient descent step
    optim.apply_gradients(zip(grad_theta, model.trainable_variables))

    return loss

# Number of training epochs
N = 5000
hist = []

# Start timer
t0 = time()

for i in range(N+1):

    loss = train_step(initial_data)

    # Append current loss to hist
    hist.append(loss.numpy())

    # Output current loss after 50 iterates
    if i%50 == 0:
        print('It {:05d}: loss = {:10.8e}'.format(i,loss))

# Print computation time
print('\nComputation time: {} seconds'.format(time()-t0))

Tensor("gradient_tape/sequential_1/dense_9/MatMul/MatMul:0", shape=(10000, 1), dtype=float64)
Tensor("gradient_tape/sequential_1/dense_9/MatMul/MatMul_2:0", shape=(10000, 1), dtype=float64)
Tensor("gradient_tape/sequential_1/dense_9/MatMul/MatMul_4:0", shape=(10000, 1), dtype=float64)
Tensor("gradient_tape/sequential_1/dense_9/MatMul/MatMul:0", shape=(10000, 1), dtype=float64)
Tensor("gradient_tape/sequential_1/dense_9/MatMul/MatMul_2:0", shape=(10000, 1), dtype=float64)
Tensor("gradient_tape/sequential_1/dense_9/MatMul/MatMul_4:0", shape=(10000, 1), dtype=float64)
It 00000: loss = 3.44662483e+00
It 00050: loss = 2.59529894e-03
It 00100: loss = 4.06112609e-05
It 00150: loss = 2.60095749e-06
It 00200: loss = 2.32749361e-06
It 00250: loss = 2.25118223e-06
It 00300: loss = 2.16990342e-06
It 00350: loss = 2.08459324e-06
It 00400: loss = 1.99644466e-06
It 00450: loss = 1.90648240e-06
It 00500: loss = 1.81559028e-06
It 00550: loss = 3.64459816e-06
It 00600: loss = 1.84726033e-05
It 00650: lo

In [49]:
model(test_data)

<tf.Tensor: shape=(281, 5), dtype=float64, numpy=
array([[-5.66515595e-05, -6.87896843e-04,  9.02783450e-04,
         1.50098322e+00,  1.08740564e+00],
       [-6.52977570e-05, -6.66592108e-04,  8.80957587e-04,
         1.50098683e+00,  1.08741565e+00],
       [-7.39438022e-05, -6.45288800e-04,  8.59133078e-04,
         1.50099044e+00,  1.08742567e+00],
       ...,
       [-2.45489208e-03,  5.17930697e-03, -5.11126452e-03,
         1.50196562e+00,  1.09014938e+00],
       [-2.46350085e-03,  5.20020883e-03, -5.13269877e-03,
         1.50196908e+00,  1.09015911e+00],
       [-2.47210951e-03,  5.22110921e-03, -5.15413156e-03,
         1.50197253e+00,  1.09016884e+00]])>

In [ ]:
model = init_model()
t_0 = tf.zeros((1,1))
u=model(t_data)
print(u)
print(tf.reshape(u[:,0],shape=(-1,1)))
u = model(t_data)
a = tf.reshape(u[:,0],shape=(-1,1))
b = tf.reshape(u[:,1],shape=(-1,1))
R = tf.reshape(u[:,2],shape=(-1,1))
theta_l =tf.reshape(u[:,3],shape=(-1,1))
theta_r = tf.reshape(u[:,4],shape=(-1,1))
yl = R*tf.sin(theta_l)+b
yr = R*tf.sin(theta_r)+b
area = cal_area_theta(a,b,d,theta_l,theta_r,R)
print(area)


In [ ]:
u=model(t_data)
print(u)
#print(t_data)
print(get_r(model, t_data))

In [ ]:
model = init_model()
t_0 = tf.zeros((1,1))
u=model(t_data)
print(u)
print(tf.reshape(u[:,0],shape=(-1,1)))
u = model(t_data)
a = tf.reshape(u[:,0],shape=(-1,1))
b = tf.reshape(u[:,1],shape=(-1,1))
R = tf.reshape(u[:,2],shape=(-1,1))
theta_l =tf.reshape(u[:,3],shape=(-1,1))
theta_r = tf.reshape(u[:,4],shape=(-1,1))
yl = R*tf.sin(theta_l)+b
yr = R*tf.sin(theta_r)+b
area = cal_area_theta(a,b,d,theta_l,theta_r,R)
print(area)

In [ ]:
model()